## **Setup**

<div class="align-center">
  <a href="https://getindexify.ai/"><img src="https://getindexify.ai/Indexify_Logo_Wordmark.svg" width="145"></a>
  <a href="https://discord.com/invite/kF8UZACA7r"><img src="https://raw.githubusercontent.com/rishiraj/random/main/Discord%20button.png" width="145"></a><br>
  Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/tensorlakeai/indexify">Github</a></i> ⭐
</div>

In [ ]:
%pip install indexify indexify-extractor-sdk indexify-langchain langchain langchain-openai

# Download Indexify Server
!curl https://getindexify.ai | sh

# Download Extractors
!indexify-extractor download tensorlake/donut-simple-invoice-parser
!indexify-extractor download tensorlake/minilm-l6

After installing the necessary libraries, download the server, and the extractors, you need to restart the runtime. Then, you have to run Indexify Server with the Extractors.

Open 2 terminals and run the following commands:

```bash
# Terminal 1
./indexify server -d

# Terminal 2
indexify-extractor join-server
```

## **Extraction Graph Setup**

1. Import the `ExtractionGraph` class from the `indexify` package.

2. Define the extraction graph specification in YAML format:
   - Set the name of the extraction graph to "invoices".
   - Define the extraction policies:
     - Use the "tensorlake/donut-invoice" extractor for invoice extraction and name it "invoice-extraction".
     - Use the "tensorlake/minilm-l6" extractor for embedding and name it "get-embeddings".
       - Set the content source for embedding to "invoice-extraction".

3. Create an `ExtractionGraph` object from the YAML specification using `ExtractionGraph.from_yaml()`.

4. Create the extraction graph on the Indexify client using `client.create_extraction_graph()`.

In [ ]:
from indexify import IndexifyClient
client = IndexifyClient()

In [ ]:
from indexify import ExtractionGraph

extraction_graph_spec = """
name: 'invoices'
extraction_policies:
   - extractor: 'tensorlake/donut-invoice'
     name: 'invoice-extraction'
   - extractor: 'tensorlake/minilm-l6'
     name: 'get-embeddings'
     content_source: 'invoice-extraction'
"""

extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
client.create_extraction_graph(extraction_graph)

## **Upload a Invoice File**

In [ ]:
import requests
req = requests.get("https://extractor-files.diptanu-6d5.workers.dev/invoice-example.jpg")

with open('invoice-example.jpg','wb') as f:
    f.write(req.content)

In [ ]:
content_id = client.upload_file("invoices", path="invoice-example.jpg")
client.wait_for_extraction(content_id)

## **What is happening behind the scenes**

Indexify is designed to seamlessly respond to ingestion events by assessing all existing policies and triggering the necessary extractors for extraction. Once the Invoice extractor completes the process of extracting texts from the document, it automatically initiates the embedding extractor to chunk the content, extract embeddings, and populate an index.

With Indexify, you have the ability to upload hundreds of Invoice files simultaneously, and the platform will efficiently handle the extraction and indexing of the contents without requiring manual intervention. To expedite the extraction process, you can deploy multiple instances of the extractors, and Indexify's built-in scheduler will transparently distribute the workload among them, ensuring optimal performance and efficiency.

### **Perform RAG**
Initialize the Langchain Retriever.

In [ ]:
from indexify_langchain import IndexifyRetriever
params = {"name": "invoices.get-embeddings.embedding", "top_k": 3}
retriever = IndexifyRetriever(client=client, params=params)

Now create a chain to prompt OpenAI with data retreived from Indexify to create a simple Q&A bot

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

Now ask any question related to the ingested Invoice

In [ ]:
chain.invoke("How much does the Camisol Top cost?")
# The Eggshell Camisol Top costs $123